# Dogs vs. Cats Image Classification Using Convolutional Neural Networks (InceptionResNetV2)¶

Download the Dogs vs Cats image catalog from Kaggle here : https://www.kaggle.com/competitions/dogs-vs-cats/data
